In [1]:
# prompt: hubungkan ke drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import os

# Load your h5 model
model = tf.keras.models.load_model('/content/drive/MyDrive/Tubes_PCD/cnn_model.h5') # Replace with your model path

# Preprocessing
IMG_WIDTH = 224  # Adjust as needed based on your dataset
IMG_HEIGHT = 224 # Adjust as needed based on your dataset
image_size = (IMG_WIDTH, IMG_HEIGHT)
BATCH_SIZE = 32  # Adjust as needed

# Load data from Google Drive folder
data_dir = '/content/drive/MyDrive/Tubes_PCD/splitted_augment_data'

filepaths = []
labels = []

for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):  # Make sure it's a directory (a class folder)
        for filename in os.listdir(class_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):  # Adjust file extensions if needed
                filepaths.append(os.path.join(class_dir, filename))
                labels.append(class_name)  # Use folder name as the label

# Create a DataFrame from the filepaths and labels
df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])

# Image data generators with augmentation for training data only
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescale validation/test data
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepaths',
    y_col='labels',
    target_size=image_size,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # Important for multi-class
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepaths',
    y_col='labels',
    target_size=image_size,
    batch_size=BATCH_SIZE,
    class_mode='categorical', # Important for multi-class
    shuffle=False  # Important for consistent predictions
)

# Get true labels and predictions
y_true = test_generator.classes  # True class labels
class_indices = test_generator.class_indices
class_labels = list(class_indices.keys())

# Make predictions
predictions = model.predict(test_generator)
y_pred_classes = np.argmax(predictions, axis=1)  # Predicted class labels

# Generate classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 1280 validated image filenames belonging to 4 classes.
Found 320 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 9s/step
Classification Report:
              precision    recall  f1-score   support

    Broccoli       0.74      1.00      0.85        80
    Capsicum       0.85      0.76      0.80        80
      Carrot       0.93      0.80      0.86        80
      Potato       0.87      0.78      0.82        80

    accuracy                           0.83       320
   macro avg       0.85      0.83      0.83       320
weighted avg       0.85      0.83      0.83       320

Confusion Matrix:
[[80  0  0  0]
 [11 61  4  4]
 [ 4  7 64  5]
 [13  4  1 62]]
